In [4]:
import numpy

from numpy.random import RandomState
from sklearn.datasets import fetch_olivetti_faces

dataset = fetch_olivetti_faces(shuffle = True, random_state = RandomState(1))
#dataset = fetch_olivetti_faces(shuffle = True, random_state = RandomState(127))
faces = dataset.data

(n_samples, n_features) = faces.shape
print(faces.shape)

# global centering
faces_mean = faces.mean(axis = 0)
faces_centered = faces - faces_mean

# local centering
faces_centered -= faces_centered.mean(axis = 1).reshape(n_samples, -1)

faces_centered.shape # |images| x |pixels|

split = int(0.25*len(faces))

test_faces = faces_centered[:split]
test_targets = dataset.target[:split]

train_faces = faces_centered[split:]
train_targets = dataset.target[split:]

(400, 4096)


# Perform Eigen Analysis

## Compute surrogate covariance matrix

In [5]:
A = train_faces
A.shape
L = A.dot(A.T)
L.shape # |images| x |images|

(300, 300)

## Compute strong eigen vectors

In [6]:
(L_eigenvalues, L_eigenvectors) = numpy.linalg.eig(L)
L_eigenvectors_strong = L_eigenvectors[:,numpy.array([True if (x > 1) else False for x in L_eigenvalues])]
L_eigenvectors_strong.shape # |images| x |strong images|

(300, 297)

## Compute eigenfaces

In [7]:
C_eigenvectors = A.T.dot(L_eigenvectors_strong).T # eigenfaces
C_eigenvectors.shape # |images| x |pixels|

eigenfaces = C_eigenvectors

## Project faces into eigenface space

In [8]:
# Project all training data into eigenface space
train_faces_projected = numpy.vstack([eigenfaces.dot(train_face) for train_face in train_faces])
train_faces_projected.shape

(300, 297)

## Perform test on faces outside our model

In [ ]:
import matplotlib.pyplot as plt

def plot_image(title, image, n_col, n_row):
    plt.figure()
    image_shape = [n_col, n_row]
    plt.title(title)
    plt.imshow(
        image.reshape(image_shape),
        cmap = plt.cm.gray,
        interpolation = 'nearest'
    )
    plt.show()

# Recogniztion

correct = 0.0
total = len(test_faces)

for test_index in range(total):
    
    test_face = test_faces[test_index]
    test_target = test_targets[test_index]
    
    test_face_projected = eigenfaces.dot(test_face) # Test is already normalized
    
    distances = numpy.array([((test_face_projected - train_face_projected)**2).sum() for train_face_projected in train_faces_projected])
    
    guess_index = distances.argmin()
    guess_face = train_faces[guess_index]
    guess_target = train_targets[guess_index]
    
    if (test_target == guess_target):
        correct += 1.0
        #print([test_target, guess_target])
    else:
        #plot_image("Test Face", test_face, 64, 64)
        #plot_image("Guess Face", guess_face, 64, 64)

print '%d%% accuracy' % int(100.0*correct/total)
print '... with %d unique people' % int(len(frozenset(dataset.target)))